<a href="https://colab.research.google.com/github/Zion-Ma/AOI-on-Wafer-Defect/blob/main/Word_Embeddings_in_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from numpy import array
from keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
# define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])


# integer encode the documents
vocab_size = 50
# encoded_docs = [one_hot(d, vocab_size) for d in docs]
# print(encoded_docs)
# pad documents to a max length of 4 words

max_length = 4
# print(encoded_docs)
# padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
# print(padded_docs)

def encoding(docs):
	encoded_docs = [one_hot(d, vocab_size) for d in docs]
	# print(encoded_docs)
	padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
	# print(padded_docs)
	return padded_docs

padded_docs = encoding(docs)
	

# define the model
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
model.summary()
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 8)              400       
                                                                 
 flatten (Flatten)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
Accuracy: 89.999998


In [2]:
s = "Could have done better"
s = one_hot(s, vocab_size)
s = pad_sequences([s], maxlen=max_length, padding='post')
w = model.predict(s)[0,0]
if w < 0.5:
	print(1-w, 'NEGATIVE')
else:
  print(w, 'POSITIVE')

1/1 [==============================] - 0s 62ms/step
0.6145768463611603 NEGATIVE


In [6]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import Flatten
# from keras.layers import Embedding

# define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])


# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
print(t.word_index)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
print(encoded_docs)
# pad documents to a max length of 4 words
max_length = max(len(item) for item in encoded_docs)
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)


# load the whole embedding into memory
embeddings_index = dict()
f = open('/content/drive/MyDrive/2022_NLP/GloVe/glove.6B.100d.txt', mode='rt', encoding='utf-8')
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

{'work': 1, 'done': 2, 'good': 3, 'effort': 4, 'poor': 5, 'well': 6, 'great': 7, 'nice': 8, 'excellent': 9, 'weak': 10, 'not': 11, 'could': 12, 'have': 13, 'better': 14}
[[6, 2], [3, 1], [7, 4], [8, 1], [9], [10], [5, 4], [11, 3], [5, 1], [12, 13, 2, 14]]
[[ 6  2  0  0]
 [ 3  1  0  0]
 [ 7  4  0  0]
 [ 8  1  0  0]
 [ 9  0  0  0]
 [10  0  0  0]
 [ 5  4  0  0]
 [11  3  0  0]
 [ 5  1  0  0]
 [12 13  2 14]]
Loaded 400000 word vectors.


In [49]:
# define model
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
model.summary()
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=1)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_30 (Embedding)    (None, 4, 100)            1500      
                                                                 
 flatten_28 (Flatten)        (None, 400)               0         
                                                                 
 dense_21 (Dense)            (None, 1)                 401       
                                                                 
Total params: 1,901
Trainable params: 401
Non-trainable params: 1,500
_________________________________________________________________
Epoch 1/50
1/1 [==============================] - 1s 611ms/step - loss: 0.6966 - acc: 0.4000
Epoch 2/50
1/1 [==============================] - 0s 9ms/step - loss: 0.6820 - acc: 0.5000
Epoch 3/50
1/1 [==============================] - 0s 7ms/step - loss: 0.6680 - acc: 0.5000
Epoch 4/50
1/1 [=======================

Accuracy: 100.000000


In [47]:
# Functional API calling

# from keras.layers import Input
# from keras import Model

# input = Input(shape = (max_length,))
# x = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)(input)
# x = Flatten()(x)
# output = Dense(1, activation='sigmoid')(x)
# model_try = Model(inputs = input, outputs = output, name = "model_try")
# model_try.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# model_try.summary()

# model_try.fit(padded_docs, labels, epochs=50, verbose=0)
# # evaluate the model
# loss, accuracy = model_try.evaluate(padded_docs, labels, verbose=0)
# print('Accuracy: %f' % (accuracy*100))

Model: "model_try"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None, 4)]               0         
                                                                 
 embedding_28 (Embedding)    (None, 4, 100)            1500      
                                                                 
 flatten_26 (Flatten)        (None, 400)               0         
                                                                 
 dense_19 (Dense)            (None, 1)                 401       
                                                                 
Total params: 1,901
Trainable params: 401
Non-trainable params: 1,500
_________________________________________________________________
Accuracy: 100.000000


In [69]:
def token_encoding(s):
  encoded_s = t.texts_to_sequences([s])
  # pad documents to a max length of 4 words
  padded_s = pad_sequences(encoded_s, maxlen=max_length, padding='post')
  return padded_s

for s in docs:
  # integer encode the documents
  padded_s = token_encoding(s)
  y_pred = model.predict(padded_s, verbose = 0)[0,0]
  print(s, end = "\t")
  if y_pred < 0.5:
    print(1-y_pred, 'NEGATIVE')
  else:
    print(y_pred, 'POSITIVE')

Well done!	0.73316574 POSITIVE
Good work	0.7211444 POSITIVE
Great effort	0.67933047 POSITIVE
nice work	0.742616 POSITIVE
Excellent!	0.7419425 POSITIVE
Weak	0.7231066226959229 NEGATIVE
Poor effort!	0.6894666850566864 NEGATIVE
not good	0.6862711608409882 NEGATIVE
poor work	0.596337229013443 NEGATIVE
Could have done better.	0.9444424845278263 NEGATIVE


In [75]:
s = "weak commitment"
padded_s = token_encoding(s)
y_pred = model.predict(padded_s, verbose = 0)[0,0]
print(s, end = "\t")
if y_pred < 0.5:
  print(1-y_pred, 'NEGATIVE')
else:
  print(y_pred, 'POSITIVE')

weak commitment	0.7231066226959229 NEGATIVE


In [77]:
! cp /content/drive/MyDrive/dis_hw.ipynb /content/